<a href="https://colab.research.google.com/github/patty-13/DEEP_LEARNING_NJIT/blob/main/HW8_DEEPLEARNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"  # specify which GPU(s) to be used
import numpy as np
import h5py
import matplotlib.pyplot as plt
import os.path
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
import random
import torch

In [ ]:
seed = 1213
random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [ ]:
# An example of load labelled data.
# The unlabelled data and test data have similar format
from google.colab import drive
drive.mount('/content/gdrive')
!gdown --id '1YfFHDr9sV-98K4bf0d89W49-73wx7m0n' --output "labelled.zip"

Mounted at /content/gdrive
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1YfFHDr9sV-98K4bf0d89W49-73wx7m0n
To: /content/labelled.zip
100% 2.67G/2.67G [00:13<00:00, 203MB/s]


In [ ]:
!unzip -q "labelled.zip" -d "./"

In [ ]:
!unzip  '/content/gdrive/MyDrive/unlabelled_data.zip'

Archive:  /content/gdrive/MyDrive/unlabelled_data.zip
   creating: unlabelled_data/
  inflating: __MACOSX/._unlabelled_data  
  inflating: unlabelled_data/file_28.hdf5  
  inflating: __MACOSX/unlabelled_data/._file_28.hdf5  
  inflating: unlabelled_data/file_45.hdf5  
  inflating: __MACOSX/unlabelled_data/._file_45.hdf5  
  inflating: unlabelled_data/file_12.hdf5  
  inflating: __MACOSX/unlabelled_data/._file_12.hdf5  
  inflating: unlabelled_data/file_53.hdf5  
  inflating: __MACOSX/unlabelled_data/._file_53.hdf5  
  inflating: unlabelled_data/file_24.hdf5  
  inflating: __MACOSX/unlabelled_data/._file_24.hdf5  
  inflating: unlabelled_data/file_32.hdf5  
  inflating: __MACOSX/unlabelled_data/._file_32.hdf5  
  inflating: unlabelled_data/file_49.hdf5  
  inflating: __MACOSX/unlabelled_data/._file_49.hdf5  
  inflating: unlabelled_data/file_48.hdf5  
  inflating: __MACOSX/unlabelled_data/._file_48.hdf5  
  inflating: unlabelled_data/file_33.hdf5  
  inflating: __MACOSX/unlabelled_data/

MAIN CODE STARTS HERE.....

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"  # specify which GPU(s) to be used
import numpy as np
import h5py
import matplotlib.pyplot as plt
import os.path
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
import random
import torch
from torch.optim.lr_scheduler import ReduceLROnPlateau
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

CUSTOM DATASET, PREPROCESSING

In [ ]:
class CustomDataset():
  def __init__(self, file_paths):
    self.file_paths = file_paths
    self.sample_size = 0
    self.index_mapping = []

    for f_p in self.file_paths:
      with h5py.File(f_p, 'r') as f:
        current_file_size = f['H_Re'].shape[0]
        self.sample_size += current_file_size
        self.index_mapping.extend([f_p, i] for i in range(current_file_size))

  def __len__(self):
    return self.sample_size

  def __getitem__(self,idx):
    file_path, local_index = self.index_mapping[idx]
    with h5py.File(file_path, 'r') as file:
      H_Re = torch.from_numpy(file['H_Re'][local_index]).float()
      H_Im = torch.from_numpy(file['H_Im'][local_index]).float()
      SNR = torch.from_numpy(file['SNR'][local_index]).float()
    return H_Re, H_Im, SNR


SELF-SUPERVISED MODEL

In [ ]:
class Encoder(nn.Module):
  def __init__(self):
    super(Encoder, self).__init__()
    self.conv1 = nn.Conv2d(56, 32, kernel_size = 3, stride = 1, padding = 1)
    self.bn1 = nn.BatchNorm2d(32)
    self.conv2 = nn.Conv2d(32, 16, kernel_size = 3, stride = 1, padding = 1)
    self.bn2 = nn.BatchNorm2d(16)

  def forward(self, x):
     x = F.relu(self.conv1(x))
     x = F.max_pool2d(x, 2)
     x = F.relu(self.conv2(x))
     x = F.max_pool2d(x, 2)
     return x
class SNRProcessor(nn.Module):
  def __init__(self):
    super(SNRProcessor, self).__init__()
    self.fc1 = nn.Linear(56 * 5, 231)

  def forward(self, x):
    x = x.view(-1, 56*5)
    x = F.relu(self.fc1(x))
    x = x.view(-1, 1, 231, 1)
    return x

class Decoder(nn.Module):
  def __init__(self):
    super(Decoder, self).__init__()
    self.conv3 = nn.Conv2d(33, 16, kernel_size=3, stride=1, padding=1)
    self.conv4 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
    self.conv5 = nn.Conv2d(32, 56, kernel_size=3, stride=1, padding=1)
    self.conv_adjust = nn.Conv2d(56, 56, kernel_size=(1, 3), stride=1, padding=0)

    # Decoder layers with batch normalization and dropout
    self.fc1 = nn.Linear(56 * 924 * 2, 280)
    self.bn1 = nn.BatchNorm1d(280)
    self.dropout1 = nn.Dropout(0.5)  # Adjust dropout rate as needed
    self.fc2 = nn.Linear(280, 56 * 5)


  def forward(self, x):

    x = F.relu(self.conv3(x))
    x = F.interpolate(x, scale_factor=2)
    x = F.relu(self.conv4(x))
    x = F.interpolate(x, scale_factor=2)
    x = torch.sigmoid(self.conv5(x))
    x = self.conv_adjust(x)
    x = x.view(x.size(0), -1)

    # Decoder layers with batch normalization and dropout
    x = F.relu(self.bn1(self.fc1(x)))
    x = self.dropout1(x)
    x = self.fc2(x)
    x = x.view(-1, 56, 5)
    return x


class Autoencoder(nn.Module):
  def __init__(self):
    super(Autoencoder, self).__init__()
    self.encoder = Encoder()
    self.snr_processor = SNRProcessor()
    self.decoder = Decoder()

  def forward(self, re, im, snr):
    # print("Input re type:", re.dtype)  # Check data type
    # print("Input im type:", im.dtype)  # Check data type
    # print("Input SNR type:", snr.dtype)  # Check data type

    encoded_re = self.encoder(re)
    encoded_im = self.encoder(im)
    processed_snr = self.snr_processor(snr)

    # print("Encoded re type:", encoded_re.dtype)  # Check data type
    # print("Encoded im type:", encoded_im.dtype)  # Check data type
    # print("Processed SNR type:", processed_snr.dtype)

    concat_data = torch.cat((encoded_re, encoded_im, processed_snr), dim = 1)
    decoded_data = self.decoder(concat_data)

    # print("Output type:", decoded_data.dtype)
    return decoded_data




SELF SUPERVISED TRAINING

In [ ]:
checkpoint_dir = '/content/checkpoints/'
os.makedirs(checkpoint_dir, exist_ok=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Is CUDA available: ", torch.cuda.is_available())

model = Autoencoder().to(device)
if torch.cuda.is_available():
    model = model

file_paths = [str(path) for path in Path('/content/unlabelled_data/').glob('*.hdf5')]

dataset = CustomDataset(file_paths)

data_loader = DataLoader(dataset,
                         batch_size=32,
                         shuffle=True,
                         num_workers = 8)

criterion = nn.SmoothL1Loss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-5)



num_epochs = 50

scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.5, min_lr=1e-6, verbose=True)
early_stopping_patience = 5
early_stopping_counter = 0
best_loss = float('inf')

# Training Loop
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for H_Re, H_Im, SNR in tqdm(data_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        # H_Re, H_Im, SNR = H_Re.to(device), H_Im.to(device), SNR.to(device)
        H_Re = H_Re.to(device).to(torch.float32)
        H_Im = H_Im.to(device).to(torch.float32)
        SNR = SNR.to(device).to(torch.float32)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(H_Re, H_Im, SNR)
        loss = criterion(outputs, SNR)  # Consider changing the loss function if needed

        # Backward and optimize
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    # Calculate average loss for the epoch
    avg_loss = epoch_loss / len(data_loader)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Avg. Loss: {avg_loss:.4f}')

    # Scheduler step and Early Stopping
    scheduler.step(avg_loss)
    if avg_loss < best_loss:
        best_loss = avg_loss
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= early_stopping_patience:
            print("Early stopping triggered.")
            break

# Save the model
    checkpoint_path = os.path.join(checkpoint_dir, f'autoencoder_epoch_{epoch+1}.pth')
    torch.save(model.state_dict(), checkpoint_path)
    print(f'Model saved to {checkpoint_path}')


Is CUDA available:  True


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Epoch 1/50: 100%|██████████| 1131/1131 [05:24<00:00,  3.49it/s]


Epoch [1/50], Avg. Loss: 24.1852
Model saved to /content/checkpoints/autoencoder_epoch_1.pth


Epoch 2/50: 100%|██████████| 1131/1131 [05:26<00:00,  3.47it/s]


Epoch [2/50], Avg. Loss: 19.5208
Model saved to /content/checkpoints/autoencoder_epoch_2.pth


...

Epoch [48/50], Avg. Loss: 15.4177
Model saved to /content/checkpoints/autoencoder_epoch_48.pth


Epoch 49/50: 100%|██████████| 1131/1131 [05:30<00:00,  3.42it/s]


Epoch [49/50], Avg. Loss: 15.4439
Model saved to /content/checkpoints/autoencoder_epoch_49.pth


Epoch 50/50: 100%|██████████| 1131/1131 [05:37<00:00,  3.35it/s]


Epoch [50/50], Avg. Loss: 15.4144
Model saved to /content/checkpoints/autoencoder_epoch_50.pth


In [ ]:
!nvidia-smi
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu120



Wed Nov 15 04:34:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

SUPERVISED MODEL

In [ ]:
class Labeled_Dataset():
    def __init__(self, file_paths):

      self.file_paths = file_paths
      self.sample_size = 0
      self.index_mapping = []

      for f_p in self.file_paths:
        with h5py.File(f_p, 'r') as f:
          current_file_size = f['H_Re'].shape[0]
          self.sample_size += current_file_size
          self.index_mapping.extend([f_p, i] for i in range(current_file_size))

    def __len__(self):
      return self.sample_size

    def __getitem__(self,idx):

      file_path, local_index = self.index_mapping[idx]
      with h5py.File(file_path, 'r') as file:
        H_Re = torch.from_numpy(file['H_Re'][local_index]).float()
        H_Im = torch.from_numpy(file['H_Im'][local_index]).float()
        SNR = torch.from_numpy(file['SNR'][local_index]).float()
        POS = torch.from_numpy(file['Pos'][local_index]).float()
      return H_Re, H_Im, SNR, POS

In [ ]:
# class PositionPredictor(nn.Module):
#     def __init__(self, trained_encoder, trained_snr_processor):
#         super(PositionPredictor, self).__init__()
#         self.encoder = trained_encoder
#         self.snr_processor = trained_snr_processor

#         # Adjust the input size according to your encoded feature size
#         # Assuming the encoder outputs 16 * 231 * 1 features per sample
#         # and the SNR processor outputs a 231-dimensional vector
#         total_feature_size = (16 * 231 * 1) * 2 + 231
#         self.fc = nn.Linear(total_feature_size, 3)  # 3 for x, y, z coordinates

#     def forward(self, H_Re, H_Im, SNR):
#         encoded_re = self.encoder(H_Re)
#         encoded_im = self.encoder(H_Im)
#         processed_snr = self.snr_processor(SNR)

#         # Flatten and concatenate the outputs
#         encoded_re_flat = encoded_re.view(encoded_re.size(0), -1)
#         encoded_im_flat = encoded_im.view(encoded_im.size(0), -1)
#         processed_snr_flat = processed_snr.view(processed_snr.size(0), -1)
#         encoded_features = torch.cat([encoded_re_flat, encoded_im_flat, processed_snr_flat], dim=1)

#         position = self.fc(encoded_features)
#         return position

import torch
import torch.nn as nn

class PositionPredictor(nn.Module):
    def __init__(self, trained_encoder, trained_snr_processor):
        super(PositionPredictor, self).__init__()
        self.encoder = trained_encoder
        self.snr_processor = trained_snr_processor

        # Adjust the input size according to your encoded feature size
        # Assuming the encoder outputs 16 * 231 * 1 features per sample
        # and the SNR processor outputs a 231-dimensional vector
        total_feature_size = (16 * 231 * 1) * 2 + 231

        # Add Batch Normalization
        self.batch_norm = nn.BatchNorm1d(total_feature_size)

        # Fully connected layer
        self.fc = nn.Linear(total_feature_size, 3)  # 3 for x, y, z coordinates

        # Dropout layer


    def forward(self, H_Re, H_Im, SNR):
        encoded_re = self.encoder(H_Re)
        encoded_im = self.encoder(H_Im)
        processed_snr = self.snr_processor(SNR)

        # Flatten and concatenate the outputs
        encoded_re_flat = encoded_re.view(encoded_re.size(0), -1)
        encoded_im_flat = encoded_im.view(encoded_im.size(0), -1)
        processed_snr_flat = processed_snr.view(processed_snr.size(0), -1)

        encoded_features = torch.cat([encoded_re_flat, encoded_im_flat, processed_snr_flat], dim=1)

        # Apply Batch Normalization
        encoded_features_bn = self.batch_norm(encoded_features)

        # Apply Dropout
        # encoded_features_dropout = self.dropout(encoded_features_bn)

        position = self.fc(encoded_features_bn)
        return position




VALIDATION

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Is CUDA available: ", torch.cuda.is_available())
# Define hyperparameters
batch_size = 32
learning_rate = 0.001
num_epochs = 20

# Split the labeled dataset into training and validation sets
file_paths = [str(path) for path in Path('/content/labelled_data/').glob('*.hdf5')]
train_file_paths, val_file_paths = train_test_split(file_paths, test_size=0.2, random_state=42)

# Create DataLoader for training and validation
train_dataset = Labeled_Dataset(train_file_paths)
val_dataset = Labeled_Dataset(val_file_paths)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Initialize the model
trained_autoencoder = Autoencoder()
#trained_autoencoder.load_state_dict(torch.load('/content/checkpoints/autoencoder_epoch_50.pth')).
trained_autoencoder.load_state_dict(torch.load('/content/autoencoder_epoch_21.pth', map_location=torch.device('cpu')))

trained_autoencoder.to(device)
trained_encoder = trained_autoencoder.encoder
trained_snr_processor = trained_autoencoder.snr_processor
supervised_model = PositionPredictor(trained_encoder, trained_snr_processor).to(device)

# Define loss function (Mean Absolute Error) and optimizer
criterion = nn.SmoothL1Loss()
optimizer = optim.AdamW(supervised_model.parameters(), lr=learning_rate, weight_decay=1e-2)
# scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.5, min_lr=1e-6, verbose=True)
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)
# Training loop
for epoch in range(num_epochs):
    supervised_model.train()
    total_loss = 0.0

    for H_Re, H_Im, SNR, POS in train_loader:
        H_Re = H_Re.to(device).to(torch.float32)
        H_Im = H_Im.to(device).to(torch.float32)
        SNR = SNR.to(device).to(torch.float32)
        POS = POS.to(device).to(torch.float32)
        # Forward pass
        predicted_position = supervised_model(H_Re, H_Im, SNR)

        # Compute the loss
        loss = criterion(predicted_position, POS)

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    scheduler.step()
    # Print training loss for this epoch
    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {total_loss / len(train_loader)}")

    # Validation loop
    supervised_model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for H_Re, H_Im, SNR, POS in val_loader:
            H_Re = H_Re.to(device).to(torch.float32)
            H_Im = H_Im.to(device).to(torch.float32)
            SNR = SNR.to(device).to(torch.float32)
            POS = POS.to(device).to(torch.float32)
        # Forward pass

            # Forward pass
            predicted_position = supervised_model(H_Re, H_Im, SNR)

            # Compute the validation loss
            val_loss += criterion(predicted_position, POS).item()

    # Print validation loss for this epoch
    #scheduler.step(val_loss)
    print(f"Validation Loss: {val_loss / len(val_loader)}")

# Save the trained model
torch.save(supervised_model.state_dict(), 'supervised_model.pth')


Is CUDA available:  True
Epoch [1/20] - Loss: 276.6199935277303
Validation Loss: 212.47847175598145
Epoch [2/20] - Loss: 123.6341940164566
Validation Loss: 53.58294069766998
Epoch [3/20] - Loss: 48.86056456963221
Validation Loss: 43.99505317211151
Epoch [4/20] - Loss: 43.92183597882589
Validation Loss: 39.97443652153015
Epoch [5/20] - Loss: 40.18163484334946
Validation Loss: 37.76792508363724
Epoch [6/20] - Loss: 39.26855502525965
Validation Loss: 36.893955171108246
Epoch [7/20] - Loss: 38.14927087227503
Validation Loss: 35.62565207481384
Epoch [8/20] - Loss: 36.5435116092364
Validation Loss: 35.46857851743698
Epoch [9/20] - Loss: 35.47769449154536
Validation Loss: 33.619841277599335
Epoch [10/20] - Loss: 34.479462722937264
Validation Loss: 33.50402253866196
Epoch [11/20] - Loss: 34.19695373376211
Validation Loss: 32.488071620464325
Epoch [12/20] - Loss: 33.55821621417999
Validation Loss: 31.95210152864456
Epoch [13/20] - Loss: 32.15605507294337
Validation Loss: 32.32102471590042
Epoch

In [ ]:
!unzip  '/content/gdrive/MyDrive/test.zip'

Archive:  /content/gdrive/MyDrive/test.zip
   creating: test/
  inflating: test/file_1.hdf5        


In [ ]:
# Dataset class for test data
class TestDataset(Dataset):

  def __init__(self, file_paths):
    self.file_paths = file_paths
    self.sample_size = 0
    self.index_mapping = []

    for f_p in self.file_paths:
      with h5py.File(f_p, 'r') as f:
        current_file_size = f['H_Re'].shape[0]
        self.sample_size += current_file_size
        self.index_mapping.extend([f_p, i] for i in range(current_file_size))

  def __len__(self):

    return self.sample_size

  def __getitem__(self,idx):

    file_path, local_index = self.index_mapping[idx]
    with h5py.File(file_path, 'r') as file:
      H_Re = torch.from_numpy(file['H_Re'][local_index]).float()
      H_Im = torch.from_numpy(file['H_Im'][local_index]).float()
      SNR = torch.from_numpy(file['SNR'][local_index]).float()

    return H_Re, H_Im, SNR
# Similar to CustomDataset but without Pos
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
supervised_model = PositionPredictor(trained_encoder, trained_snr_processor)
model_path = '/content/supervised_model.pth'
state_dict = torch.load(model_path, map_location=torch.device('cpu'))
supervised_model.load_state_dict(state_dict)
supervised_model.to(device)
supervised_model.eval()

# Create a DataLoader for the test data
test_file_paths = [str(path) for path in Path('/content/test/').glob('*.hdf5')]
test_data_loader = DataLoader(TestDataset(test_file_paths), batch_size=1)

# Predict positions
predictions = []
with torch.no_grad():  # Disable gradient calculation
    for H_Re, H_Im, SNR in test_data_loader:
        H_Re = H_Re.to(device).to(torch.float32)
        H_Im = H_Im.to(device).to(torch.float32)
        SNR = SNR.to(device).to(torch.float32)

        # Predict positions using the supervised model
        outputs = supervised_model(H_Re, H_Im, SNR)

        # Store predictions, modify this depending on the output format of your model
        predictions.append(outputs.detach().cpu().numpy())

# Flatten the predictions list if it's nested
flat_predictions = [item for sublist in predictions for item in sublist]

# Create and save the submission CSV file
submission_df = pd.DataFrame(flat_predictions, columns=['x', 'y', 'z'])
submission_df.insert(0, 'id', range(0, len(submission_df)))
submission_df.to_csv('/content/submission_pranav.csv', index=False)



SAVING RESULTS